# Legal Studies 123 Final Project #

## Gentrification & Crime in New York City Analysis ##

### Project Members: Abdul Choudhry, Sammy Chean-Udell, Christian Gutierrez, Louisa Ng ###
### Group 10 ###

In [1]:
# standard imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import folium
import json
import random
from random import sample
from folium.plugins import HeatMap, HeatMapWithTime
import geojson
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

ModuleNotFoundError: No module named 'geopandas'

## Part 1: Exploratory Data Analysis (EDA) ##

In [ ]:
# sampling 100,000 rows of the large NYPD arrests dataset for efficiency
arrests = pd.read_csv("NYPD_arrests_sampled100000.csv").drop(columns = ["Unnamed: 0"])
arrests

**In order to initially perform our exloratory data analysis, our first step was to take a random sample of 100,000 rows of the larger New York City arrest dataset, which had about one million rows of data.**

In [ ]:
arrests["year"] = arrests["ARREST_DATE"].str.extract("\d{2}\/\d{2}\/(\d{4})")
arrests["year"].value_counts()

In [ ]:
arrests.isnull().sum()

In [ ]:
# Lon_Lat column contains string type objects, but we want a POINT geometry object 
type(arrests["Lon_Lat"].iloc[0])

In [ ]:
# converts Lon_lat column from str to POINT objects
arrests["Point"] = gpd.GeoSeries.from_wkt(arrests["Lon_Lat"]) 
arrests = gpd.GeoDataFrame(arrests, geometry="Point", crs={"init": "epsg:4326"})
arrests.head(1)

In [ ]:
arrests

In [ ]:
arrests.isnull().sum()

In [ ]:
null_columns = ["PD_CD", "PD_DESC", "KY_CD", "OFNS_DESC", "LAW_CODE", "LAW_CAT_CD", "X_COORD_CD", 
               "Y_COORD_CD", "Latitude", "Longitude", "Lon_Lat", "Point"]
arrests = arrests.dropna()

# for i in range(len(null_columns)):
#     arrests.drop(arrests.index[arrests[null_columns[i]] == 0], inplace = True)

# arrests.drop(arrests.index[arrests["PD_CD"] == 0], inplace = True)
# arrests.drop(arrests.index[arrests["PD_DESC"] == 0], inplace=True)
# arrests.drop(arrests.index[arrests["KY_CD"] == 0], inplace=True)
# arrests.drop(arrests.index[arrests["OFNS_DESC"] == 0], inplace=True)
# arrests.drop(arrests.index[arrests["LAW_CODE"] == 0], inplace=True)
# arrests.drop(arrests.index[arrests["LAW_CAT_CD"] == 0], inplace=True)
arrests

In [ ]:
arrests.columns

In [ ]:
arrests.isnull().sum()

In [ ]:
sum(arrests["Lon_Lat"].isna())

**Below creates an outline of nyc neighborhoods using different data (this data has neighborhood names within the geojson file, which can be useful for later).**


In [ ]:
# Convert text file to JSON 
with open('nyc_neighborhood_outlines2_raw.txt') as json_file:     # name of the text file
    data2 = json.load(json_file)

# Convert JSON to GeoJson 
from geojson import dump
with open('nyc_neighborhood_outlines2.geojson', mode='w') as f:   # name of the output geojson file
   dump(data2, f)

nyc_neigh_outlines_json2 = json.load(open("nyc_neighborhood_outlines2_raw.txt"))

nyc_coords = [40.6955, -73.9509]
m2 = folium.Map(nyc_coords, zoom_start=11)

folium.GeoJson(nyc_neigh_outlines_json2, style_function=lambda feature: {
    'fillColor': '#ffff00',
    'fillOpacity': 0,
    'color': 'black',        
    'weight': 1
    }).add_to(m2)

m2

**Below uses the same geoJSON data as the plotted outline above and converts that data into a table. We can extract the different names of the neighborhoods from the table below, since we can't do it directly from the map.**

In [ ]:
outlines_df = gpd.read_file("nyc_neighborhood_outlines2_raw.txt")
outlines_df = outlines_df[["neighborhood", "geometry"]]
outlines_df.head(3)

**Spatial joins table of arrests with the outlines (basically creates a new column in the arrests df that adds the name of the neighborhood of where the arrest occured). We now have an arrests table with both a POINT and the neighborhood that the point falls under.**

In [ ]:
if "neighborhood" not in list(arrests.columns):
    arrests = arrests.sjoin(outlines_df, how="inner")
arrests

In [ ]:
arrests_columns = list(arrests.columns)
arrests_columns

In [ ]:
arrests.groupby("neighborhood")["Point"].count()

**Loading in the New York City evictions dataset. This dataset serves as our initial gentrification factor that we will analyze in NYC. In other words, for the purpose of our EDA, we aim to analyze the relationship between crime (aka the arrests data that we took a sample of) and evictions in New York City since eviction data tells us which regions in the city are more or less gentrified over the years.**

In [ ]:
NY_evictions = pd.read_csv("Evictions.csv")
NY_evictions

In [ ]:
NY_evictions_new = NY_evictions[["Executed Date", "Residential/Commercial", "BOROUGH", 
                                "Eviction Postcode", "Eviction/Legal Possession", "Latitude", 
                                "Longitude", "Council District", "BIN", "BBL"]].dropna()
NY_evictions_new

In [ ]:
ev_g = pd.DataFrame(NY_evictions_new.groupby('BOROUGH').size()).rename(columns = {0: 'Count'}).reset_index()
ev_g['PROP'] = [i/sum(ev_g['Count']) for i in ev_g['Count']]

plt.figure(figsize = (15, 10))
sns.barplot(x = 'BOROUGH', y = 'PROP', data = ev_g)
plt.ylabel('Proportion')
plt.xlabel('Borough')
plt.title('Proportion of Evictions Across 5 Boroughs')
plt.show()

**Created a new table with three columns: Borough, Count, and Proportion. Count is the number of times each Borough appears in the original table and Proportion is the Proportion of each count to the total number of rows in the original table. The above graph shows viewers the proportion of evictions across the five boroughs of NYC. As we can see, the Bronx borough has the highest proportion of evictions over the years.**

In [ ]:
residential_versus_commercial = NY_evictions_new.groupby("Residential/Commercial")["Executed Date"].count()
residential_versus_commercial.plot(kind = "bar", figsize = (8, 6))
plt.xlabel("Category")
plt.ylabel("Number of Evictions")
plt.xticks(rotation = 360)
plt.suptitle("Number of Residential vs. Commercial Evictions")
plt.show()

**The above plot shows the number of residential versus commercial evictions that occurred throughout the years in NYC. Clearly, the number of residential evictions outweigh the commercial ones and this is important to note because individuals from historically lower socioeconomic backgrounds and marginalized races live in residential areas and in general it is more likely for individuals to be evicted rather than commercial businesses.**

In [ ]:
# Visualizing the number of evictions per borough in 2019

NY_evictions_new["year"] = NY_evictions_new["Executed Date"].str.extract("\d{2}\/\d{2}\/(\d{4})")
evictions_2019 = NY_evictions_new[NY_evictions_new["year"] == "2019"]
cleaned = pd.DataFrame(evictions_2019.groupby("BOROUGH").size()).rename(columns = {0: "count"}).reset_index()

arrests_19 = arrests[arrests["year"] == "2019"]

cleaned_arrests = pd.DataFrame(arrests_19.groupby("ARREST_BORO").size()).rename(columns = {0: "count"}).reset_index()

ax = plt.subplots(figsize = (15, 10))
 
ax = sns.barplot(x=cleaned["BOROUGH"], y=cleaned["count"], color = 'yellow')
ax = sns.barplot(x=cleaned_arrests["ARREST_BORO"], y=cleaned_arrests["count"], color = 'black')
 
ax.set(xlabel="Borough", ylabel="Number of Arrests or Evictions", 
       title = "Number of Evictions and Arrests per Borough in 2019")
ax.set_xticklabels(["Bronx",'Brooklyn','Manhattan','Queens', "Staten Island"])

plt.legend(labels = ["Evictions (Yellow)", "Arrests (Black)"], prop = {"size": 15})
plt.xticks()
plt.show()

**According to the layered bar plot above, viewers are able to see the relationship between the number of arrests and number of evictions per borough in NYC in the year 2019. We decided to analyze this relationship in the year 2019 because it was the last unbiased year before the pandemic began, so it did not skew or visualization. We can see that there were far more evictions (yellow) than arrests or crimes (black) in 2019 per borough; however, can make the conclusion that both the Brooklyn and Manhattan boroughs have the highest number of evictions and arrests in 2019 compared to the rest of the boroughs.**

In [ ]:
arrests_copy = arrests
arrests_copy["ARREST_BORO"] = arrests_copy["ARREST_BORO"].replace({"B": "BRONX", "S": "STATEN ISLAND", 
                                                                   "K": "BROOKLYN", "M": "MANHATTAN", 
                                                                   "Q": "QUEENS"}).rename({"ARREST_BORO": "BOROUGH"})
arrests_copy = arrests_copy.rename(columns = {"ARREST_BORO": "BOROUGH"})
arrests_copy

In [ ]:
arrests2 = arrests
arrests2['ARREST_BORO'] = arrests['ARREST_BORO'].replace({'B': 'BRONX', 'S': 'STATEN ISLAND', 
                                                          'K': 'BROOKLYN' , 'M': 'MANHATTAN', 'Q': 'QUEENS'})#.rename({'ARREST_BORO': 'BOROUGH'})                                                
arrests2 = arrests2.rename(columns = {'ARREST_BORO' : 'BOROUGH'})
arrests2['ONES'] = np.ones(len(arrests2))


grouped = arrests2.groupby(['year', 'BOROUGH']).sum().reset_index()
# grouped["year"] = list(arrests2["year"].values)

grouped

In [ ]:
plt.figure(figsize = (15, 10))
sns.lineplot(x = "year", y = 'ONES', data = grouped, hue = 'BOROUGH')
plt.ylabel('Number of Crimes Committed')
plt.xlabel('Year')
plt.title('Number of Crimes Committed Across All 5 Boroughs From 2006-2020')
plt.show()

**Plotted above is the number of crimes committed over the last 14 years across all 5 boroughs. The ones column represents a counter for each row so that, when summing in the grouby step, each borough will have an accurate number of crimes per year. Additionally, we can see that all five boroughs share the same downward trend as a function of time. We can also note that Staten Island has a significantly lower number of crimes compared to the rest of the boroughs and we believe this is because of its low population size compared to other boroughs.**

## Analyzing Arrests and Evictions by Race ##

**In this part, we filtered the arrest data by PERP_RACE and neighborhood and took the count of each (PERP_RACE, neighborhood) pair in the race_count table. The Choropleth map has 6 layers (one for each race) and you can select and delect these layers using LayerControl on the top right of the map.**

In [ ]:
arrests.columns

In [ ]:
arrests["PERP_RACE"].value_counts().drop(index=["UNKNOWN"])

In [ ]:
# drop "UNKNOWN" race because we want to focus on known races
races_list = arrests["PERP_RACE"].value_counts().drop(index=["UNKNOWN"]).index.to_list()
races_list

In [ ]:
perp_race = arrests["PERP_RACE"].value_counts()

In [ ]:
neighborhoods = arrests["neighborhood"].value_counts()

In [ ]:
race_count = arrests[["neighborhood", "PERP_RACE"]].value_counts().reset_index()
race_count = race_count[race_count["PERP_RACE"].isin(races_list)].reset_index().rename(columns={"PERP_RACE":"race", 0:"count"})
race_count

In [ ]:
race_count[race_count["neighborhood"] == "6"].shape[0]

In [ ]:
neighborhood_names = outlines_df["neighborhood"].value_counts().index.tolist()

for i in neighborhood_names:
    all_i = race_count[race_count["neighborhood"] == i]
    for r in races_list:
        row = all_i[all_i["race"] == r]
        if (all_i.shape[0] == 0) or (row.shape[0] == 0):
            row = pd.DataFrame([[i, r, 0]], columns=["neighborhood", "race", "count"])
            race_count = pd.concat([race_count, row])
            
race_count.sort_values(["neighborhood", "race"], ascending=[False, False])

**The above table shows the count of each race per neighborhood that appears in the sampled arrests dataset.**

In [ ]:
NY_evictions_new

In [ ]:
eviction_lats = list(NY_evictions_new["Latitude"].values)
eviction_lons = list(NY_evictions_new["Longitude"].values)
eviction_coordinates = [list(x) for x in zip(eviction_lats, eviction_lons)]
eviction_coordinates_sample = random.sample(eviction_coordinates, 500)
neighborhood_geojson = json.load(open("nyc_neighborhood_outlines2_raw.txt"))

bins = np.arange(0, 3501, 500)
num_legends = 0
nyc_coords = [40.6955, -73.9509]

map_by_race = folium.Map(nyc_coords, zoom_start=11)
for r in races_list:
    c = folium.Choropleth(
        geo_data = neighborhood_geojson,
        data = race_count[race_count["race"] == r],
        columns = ["neighborhood", "count"],
        key_on = "feature.properties.neighborhood",
        fill_color = "Greens",
        fill_opacity = 0.9,
        line_opacity = 0.5,
        legend_name = "Arrest Counts",
        name = r
    )
    
    for key in c._children:
        if num_legends == 0:
            num_legends += 1
            break
        elif key.startswith("color_map"):
            del(c._children[key])
            
    c.add_to(map_by_race)
    
for i in range(len(eviction_coordinates_sample)):
    ## EVICTION INCIDENTS ARE YELLOW ## 
               folium.CircleMarker(eviction_coordinates_sample[i], icon = folium.Icon(color = "black"),
                                                                    radius = 4, 
                                                                    color = "black",
                                                                    fill_color = "yellow",
                                                                    fill = True,
                                                                    fill_opacity = 1).add_to(map_by_race)
    
    
folium.LayerControl().add_to(map_by_race)
    
map_by_race

**The layered choropleth map above shows viewers the relationship between gentrification (specifically eviction) and crime in New York City. As we can see above, viewers are able to adjust the layered choropleth map by race and analyze which neighborhood in NYC has the most crimes by race. Also, the more green a neighborhood is the more crimes it contains and depending on the various races that a user has selected he or she is able to analyze which regions of NYC have the most evictions by race. Additionally, the yellow dots on the map represent individual eviction incidents and our visualization shows that most evictions occur in the Bronx borough which validates the findings of our layered bar plot in previous cells. Lastly, each yellow dot is taken from a random sample of eviction coordinate pairs in order to prevent the choropleth map from overloading with too many data points.**

In [ ]:
heatmap_obj = folium.Map(location = nyc_coords, zoom_start = 10)
heatmap = HeatMap(eviction_coordinates_sample, radius = 17).add_to(heatmap_obj)
heatmap_obj

**According to the heatmap above, we are able to confirm our findings from the layered choropleth map above and see that most of the data (which consist of the eviction incidents) are clusted around the Bronx borough, which is illustrated by the "red" segment on the top of the heatmap.**

In [ ]:
def getCoordinates(df, borough):
    """
    Function that takes in the NYC Eviction or crime dataset and
    one of the five boroughs in NYC and returns a sample of
    the latitude/longitude pairs for that specific borough.
    """
    eviction_df = df[df["BOROUGH"] == borough]
    borough_lat = list(eviction_df["Latitude"].values)
    borough_lon = list(eviction_df["Longitude"].values)
    borough_lat_lon_pairs = [list(x) for x in zip(borough_lat, borough_lon)]
    return random.sample(borough_lat_lon_pairs, 100)

brooklyn_coords = getCoordinates(NY_evictions_new, "BROOKLYN")
bronx_coords = getCoordinates(NY_evictions_new, "BRONX")
staten_island_coords = getCoordinates(NY_evictions_new, "STATEN ISLAND")
queens_coords = getCoordinates(NY_evictions_new, "QUEENS")
manhattan_coords = getCoordinates(NY_evictions_new, "MANHATTAN")

folium_obj = folium.Map(location = nyc_coords, zoom_start = 11)

for i in range(len(brooklyn_coords)):
    ## BROOKLYN IS BLACK ## 
               folium.CircleMarker(brooklyn_coords[i], icon = folium.Icon(color = "black"),
                                                                    radius = 4, 
                                                                    color = "black",
                                                                    fill_color = "black",
                                                                    fill = True,
                                                                    fill_opacity = 1).add_to(folium_obj)
        
for i in range(len(bronx_coords)):
    ## BRONX IS RED ##
             folium.CircleMarker(bronx_coords[i], icon = folium.Icon(color = "black"),
                                                                    radius = 4, 
                                                                    color = "black",
                                                                    fill_color = "red",
                                                                    fill = True,
                                                                    fill_opacity = 1).add_to(folium_obj)
    
for i in range(len(staten_island_coords)):
    ## STATEN ISLAND IS BLUE ##
               folium.CircleMarker(staten_island_coords[i], icon = folium.Icon(color = "black"),
                                                                    radius = 4, 
                                                                    color = "black",
                                                                    fill_color = "blue",
                                                                    fill = True,
                                                                    fill_opacity = 1).add_to(folium_obj)
for i in range(len(queens_coords)):
    ## QUEENS IS GREEN ##
               folium.CircleMarker(queens_coords[i], icon = folium.Icon(color = "black"),
                                                                    radius = 4, 
                                                                    color = "black",
                                                                    fill_color = "green",
                                                                    fill = True,
                                                                    fill_opacity = 1).add_to(folium_obj)
        
for i in range(len(manhattan_coords)):
    ## MANHATTAN IS YELLOW ##
               folium.CircleMarker(manhattan_coords[i], icon = folium.Icon(color = "black"),
                                                                    radius = 4, 
                                                                    color = "black",
                                                                    fill_color = "yellow",
                                                                    fill = True,
                                                                    fill_opacity = 1).add_to(folium_obj)
folium_obj

**The above plot shows viewers the number of evictions per borough in New York City while each borough is color-coded a specific color. Brooklyn is Black, Bronx is Red, Staten Island is Blue, Queens is Green, and Manhattan is Yellow. Additionally, each point on the folium map is taken from a random sample of coordinate pairs in order to prevent the map from overloading with too many data points.**

In [ ]:
arrest_lats = list(arrests["Latitude"].values)
arrest_lons = list(arrests["Longitude"].values)
arrest_coordinates = [list(x) for x in zip(arrest_lats, arrest_lons)]
arrest_coordinates_sample = random.sample(arrest_coordinates, 500)

In [ ]:
folium_obj = folium.Map(location = nyc_coords, zoom_start = 11)

for i in range(len(arrest_coordinates_sample)):
    ## ARRESTS ARE YELLOW ## 
               folium.CircleMarker(arrest_coordinates_sample[i], icon = folium.Icon(color = "black"),
                                                                    radius = 4, 
                                                                    color = "black",
                                                                    fill_color = "yellow",
                                                                    fill = True,
                                                                    fill_opacity = 1).add_to(folium_obj)
for i in range(len(eviction_coordinates_sample)):
    ## EVICTIONS ARE RED ## 
               folium.CircleMarker(eviction_coordinates_sample[i], icon = folium.Icon(color = "black"),
                                                                    radius = 4, 
                                                                    color = "black",
                                                                    fill_color = "red",
                                                                    fill = True,
                                                                    fill_opacity = 1).add_to(folium_obj)
folium_obj   

**This final map above shows viewers our underlying hypothesis for our project. Essentially, this map shows the relationship between crime and gentrification (specifically eviction) in New York City. In our proposal we hypothesized how eviction is one of the most prevalent factors of gentrification and can see that both eviction and arrests are clustered together in specific regions of New York City such as the Bronx. Later on in our analysis, we hope to analyze gentrification and eviction displacements as a function of time and predict gentrification by crime. We may also explore the "racial tipping point" where an area's non-white population reaches a certain threshold such that white flight exists, which is a movement of white residents to the outskirts of urban areas or regions where the "racial tipping point" occurrs.**

## Part 2: Modeling & Prediction ##

In [ ]:
NY_evictions_new.head(3)

In [ ]:
arrests.head(3)

In [ ]:
pd.set_option('display.max_columns', None)
# add month column to arrest table 
arrests["month"] = arrests["ARREST_DATE"].str.extract(r"(\d{2})\/\d{2}\/\d{4}").astype(int)

# change data type of "year" column from string to int
arrests["year"] = arrests["year"].astype(int)
arrests.head(3)

In [ ]:
arrests.columns

In [ ]:
# adds "crime_count" column to arrests table, where crime count represents
# the total number of crimes per month per year
crime_count = arrests[["year", "month", "neighborhood"]].value_counts().reset_index().rename(columns={0:"crime_count"})
arrests = arrests.merge(crime_count, on=["year", "month", "neighborhood"])
arrests

In [ ]:
NY_evictions_new.rename(columns = {"BOROUGH": "ARREST_BORO"}, inplace = True) # unnecessary to rename this column
NY_evictions_new

In [ ]:
# our ultimate goal is to join eviction data with ny arrest data on neighborhoods, so we will 
# place each eviction in a neighborhood based on latitude and longitude columns

NY_evictions_new = gpd.GeoDataFrame(NY_evictions_new, geometry=gpd.points_from_xy(
    NY_evictions_new.Longitude, NY_evictions_new.Latitude))
nyc_outlines_table = gpd.read_file("nyc_neighborhood_outlines2.geojson")

if "neighborhood" not in list(NY_evictions_new.columns):
    NY_evictions_new = NY_evictions_new.sjoin(nyc_outlines_table)
NY_evictions_new

In [ ]:
# add "month" column to NY_evictions new 
NY_evictions_new["month"] = NY_evictions_new["Executed Date"].str.extract(r"(\d{2})\/\d{2}\/\d{4}").astype(int)

# convert year column to int values 
NY_evictions_new["year"] = NY_evictions_new["year"].astype(int)

# create counts table of number of evictions based on month, year, borough, and residential/commericial 
evictions_count = NY_evictions_new[["month", "year", "neighborhood", "Residential/Commercial"]].value_counts().reset_index().rename(columns={0:"eviction_count"})

evictions_count

In [ ]:
# merge eviction counts to arrests table on the month, year, and ny neighborhood
arrests = arrests.merge(evictions_count, on=["month", "year", "neighborhood"])
print(arrests.columns)
arrests.head(1)

In [ ]:
# import NYC housing data, clean and merge with arrests table based on date, month, and neighborhood 

ny_housing = pd.read_csv("Housing_New_York_Units_by_Building.csv")
ny_housing.head(1)

In [ ]:
# create new month and year columns

ny_housing["month"] = ny_housing["Project Start Date"].str.extract(r"(\d{2})\/\d{2}\/\d{4}").astype(int)
ny_housing["year"] = ny_housing["Project Start Date"].str.extract(r"\d{2}\/\d{2}\/(\d{4})").astype(int)

In [ ]:
# runs following code once
# code does a spatial join of ny_housing data with corresponding neighborhoods

if "neighborhood" not in list(ny_housing.columns):
    ny_housing = gpd.GeoDataFrame(ny_housing, geometry=gpd.points_from_xy(
    ny_housing.Longitude, ny_housing.Latitude))
    ny_housing = ny_housing.sjoin(nyc_outlines_table)

ny_housing.head()

In [ ]:
ny_housing.columns

In [ ]:
ny_income_housing_types = ['Extremely Low Income Units', 'Very Low Income Units', 'Low Income Units', 
                    'Moderate Income Units', 'Middle Income Units', 
                    "Other Income Units", "Studio Units", "1-BR Units", "2-BR Units", 
                    "3-BR Units", "4-BR Units", "5-BR Units", "6-BR+ Units", 
                    "Unknown-BR Units", "Counted Rental Units", "Counted Homeownership Units", 
                    "All Counted Units", "Total Units"]

# ny_housing_counts = ny_housing[["month", "year", "neighborhood"] + ny_income_housing_types].value_counts().reset_index(
# ).rename(columns={0:"count"})
# ny_housing_counts

ny_housing_counts = ny_housing.groupby(["month", "year", "neighborhood"]).sum().reset_index()
ny_housing_counts = ny_housing_counts[["month", "year", "neighborhood"] + ny_income_housing_types]
ny_housing_counts

In [ ]:
# RUN MERGE ONCE

arrests = arrests.merge(ny_housing_counts, on=["month", "year", "neighborhood"])
arrests

In [ ]:
arrests.columns

In [ ]:
arrests.isnull().sum()

In [ ]:
arrests[["X_COORD_CD", "Y_COORD_CD", "AGE_GROUP", "PERP_SEX", "PERP_RACE"]]

In [ ]:
arrests_quantitative = arrests[["PD_CD", "KY_CD", "ARREST_PRECINCT", "month", "year",
                               "eviction_count", "X_COORD_CD", "Y_COORD_CD", 
                               "Latitude", "Longitude","Extremely Low Income Units", 
                               "Very Low Income Units", "Low Income Units", "Moderate Income Units", 
                               "Middle Income Units", "Other Income Units", 
                                "Studio Units", "1-BR Units", "2-BR Units", 
                                "3-BR Units", "4-BR Units", "5-BR Units", "6-BR+ Units", 
                                "Unknown-BR Units", "Counted Rental Units", "Counted Homeownership Units", 
                                "All Counted Units", "Total Units"]]
arrests_quantitative

In [ ]:
# we are trying to predict the number of crimes per month per year in NYC so predicted y variable is assigned
# to this quantity. 

# NOTE: the "crime_count" column is NOT in the "arrests_quantitative" dataframe because this will serve as 
# our training dataset, and we do not want to include the predicted column (aka "crime_count") in the training data. 

y = list(arrests["crime_count"].values)

# standardizing the features in the crimes_df dataset so that they have zero mean and unit variance 
scaler = StandardScaler()
arrests_df_standardized = pd.DataFrame(scaler.fit_transform(arrests_quantitative.values), 
                                      columns=arrests_quantitative.columns, index=arrests_quantitative.index)
arrests_df_standardized.head(20)

In [ ]:
# initial train/test split

X_train, X_test, y_train, y_test = train_test_split(arrests_df_standardized, y, train_size=0.80, test_size=0.20)

# further partitioning the data into a training, test, and validation set.

X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, train_size = 0.70, test_size=0.30)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
arrests_quant_with_crime = arrests[["PD_CD", "KY_CD", "ARREST_PRECINCT", "month", "year",
                               "eviction_count", "crime_count", "X_COORD_CD", "Y_COORD_CD", 
                               "Latitude", "Longitude","Extremely Low Income Units", 
                               "Very Low Income Units", "Low Income Units", "Moderate Income Units", 
                               "Middle Income Units", "Other Income Units", 
                                "Studio Units", "1-BR Units", "2-BR Units", 
                                "3-BR Units", "4-BR Units", "5-BR Units", "6-BR+ Units", 
                                "Unknown-BR Units", "Counted Rental Units", "Counted Homeownership Units", 
                                "All Counted Units", "Total Units"]]
arrests_quant_with_crime

In [ ]:
arrests_quant_with_crime_standardized = pd.DataFrame(scaler.fit_transform(arrests_quant_with_crime.values), 
                                      columns=arrests_quant_with_crime.columns, index=arrests_quant_with_crime.index)

arrests_quant_with_crime_standardized

In [ ]:
correlation_df = arrests_quant_with_crime_standardized.corr()

# NOTE: all values in the df below are between -1 and 1.
# In other words, -1 < r < 1, where r is the correlation coefficient of the features we plan to use 
# prior to any feature selection. 

correlation_df

In [ ]:
plt.figure(figsize = (20, 18))
sns.heatmap(correlation_df, annot = True)
plt.title("Correlation Matrix for Gentrification Features Prior to Feature Selection", fontdict = {'fontsize': 27})
plt.show()

In the above heatmap shows the correlation between the features we plan to feed into our model BEFORE we do any feature selection techniques and choose our preconceived "best" gentrification features to predict number of crimes per month per year in NYC. We can already see above that the "crime_count" feature has a correlation coefficient of 0.23 with respect to the "eviction_count" feature, which is just one feature of gentrification we are analyzing. 

In [ ]:
target_corr = abs(correlation_df["crime_count"])
target_corr

In [ ]:
decent_features = list(target_corr[target_corr > 0.05].index)
decent_features.remove("crime_count")
decent_features

In order to build an accurate model, we needed to clean our data. Initially, we dropped all null values from our tables. Although dropping null values runs the risk of losing patterns hidden in the data, we felt confident that no meaningful pattern existed within the null values for the sake of our goal. Our resulting main dataframe of New York arrest reports was stored in a variable called ‘arrests’. We merge the arrests table with two other imported datasets in order to provide more potential features to choose from during our feature selection process. The first dataset contained data on residential and commercial evictions in New York starting from 2017. We converted the Latitude and Longitude columns into point geometry objects in order to do a spatial join with geojson data that places each eviction property in its corresponding New York neighborhood. After, we extracted the month and year of each eviction from the date column and converted to values to integers. With the resulting New York evictions table, we grouped the table by year, month, and neighborhood using the value_counts method to get the total number of evictions per (month, year, neighborhood) combination and merged the resulting dataframe to the arrests table on the same columns. The second dataset contained data on construction of housing projects in New York from 2014 to 2021. Similarly, we placed each property in its corresponding neighborhood using a spatial join, extracted the month and year values from the date column, and created a counts table based on the different (month, year, neighborhood) combinations. We merged this with the arrests table. Furthermore, in order to ensure that all features would be treated equally by our feature selection methods, we had to standardize them. Standardizing our variables made it possible for our model to consider all attributes on a common scale, allowing it to differentiate between features based solely on their influence on our predicted variable, arrest counts. This allowed for our model to accurately weight each feature against the influence it holds over the feature we are predicting. Lastly, before training our data on different potential models, it was important to discern the most influential features. To do so, we used a correlation matrix and used a cutoff of 0.05 to weed out the least important features. 

In [ ]:
def rmse(pred, actual):
    return np.sqrt(np.mean((pred - actual) ** 2))

**Ordinary Least Squares regression estimates relationships between independent variables and dependent variables. In order to do this, the model reduces the sums of squares in the difference between observed and predicted values of the dependent variables, generating singular one dimensional lines of best fit. Linear regression is an incredibly efficient regression method, as it is easy to train and easy to adjust when encountering issues due to overfitting. However, Linear Regression is limited by myriad obstacles, inluding its assumption of a linear relationship between independent and dependent variable. This is problematic as data based in real life, such as our gentrification data, is much more complex. Furthermore, if the data has a larger number of parameters than observations, OLS is very likely to overfit.**

In [ ]:
lin_model = LinearRegression()
lin_model_fit = lin_model.fit(X_train, y_train)
lin_model_pred = lin_model_fit.predict(X_train)

plt.figure(figsize = (10, 8))
plt.scatter(y_train, lin_model_pred)
plt.xlabel("Actual Value")
plt.ylabel("Predicted Value")
plt.suptitle("Linear Model (OLS)")
plt.show()

print("Linear Regression Model RMSE:", rmse(lin_model_pred, y_train))
print("Linear Regression Model Training Accuracy:", lin_model_fit.score(X_train, y_train))

In [ ]:
linear_cross_val_pred = cross_val_predict(lin_model, arrests_df_standardized, y, cv = 4)
r2_score(y, linear_cross_val_pred)

In [ ]:
new_lin_model = LinearRegression()
new_lin_model_fit = new_lin_model.fit(X_train[decent_features], y_train)
new_lin_model_pred = new_lin_model_fit.predict(X_train[decent_features])

plt.figure(figsize = (10, 8))
plt.scatter(y_train, new_lin_model_pred)
plt.xlabel("Actual Value")
plt.ylabel("Predicted Value")
plt.suptitle("New Linear Model (OLS)")
plt.show()

print("New Linear Regression Model RMSE:", rmse(new_lin_model_pred, y_train))
print("New Linear Regression Model Training Accuracy:", new_lin_model_fit.score(X_train[decent_features], y_train))

In [ ]:
# Using one of the feature selection methods from Lab 14
recursive_feature_elimination = RFE(lin_model, n_features_to_select = 7)
recursive_feature_elimination.fit(X_train, y_train)

In [ ]:
print(recursive_feature_elimination.support_)

"The feature ranking, such that ranking_[i] corresponds to the ranking position of the i-th feature. Selected (i.e., estimated best) features are assigned rank 1" (Marshall, Lab 14). 

In [ ]:
print(recursive_feature_elimination.ranking_)

In [ ]:
rfe_pred = recursive_feature_elimination.predict(X_train)
print("RFE Model RMSE:", rmse(rfe_pred, y_train))
print("RFE Model Training Accuracy:", recursive_feature_elimination.score(X_train, y_train))

**Ridge Regression produces a predictive model by creating a colinear equation weighting features by the influence it holds over our predicted variable. A feature with a heavier weight is seemingly much more influential, while a feature with a lighter weight impacts the predicted variable less. Ridge Regression will help us create an accurate predictive model because it works very well when fed with multiple features. Even though Ridge Regression is a good choice of model, it is important to acknowledge its disadvantages. For instance, a feature may, in code, seem to have no bearing over our predicted variable, yet it could have a larger societal meaning that is not be encoded in the data. This would mean that Ridge Regression assigns a lighter weight to the feature, even though it holds great influence over the predicted variable.**

In [ ]:
ridge = Ridge()
ridge_model = ridge.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_train)

plt.figure(figsize = (8, 6))
plt.scatter(y_train, ridge_pred)
plt.xlabel("Actual Value")
plt.ylabel("Predicted Value")
plt.suptitle("Ridge Model")
plt.show()

print("Ridge RMSE:", rmse(ridge_pred, y_train))
print("Ridge Training Accuracy:", ridge_model.score(X_train, y_train))

In [ ]:
new_ridge = Ridge()
new_ridge_model = new_ridge.fit(X_train[decent_features], y_train)
new_ridge_pred = new_ridge_model.predict(X_train[decent_features])

plt.figure(figsize = (8, 6))
plt.scatter(y_train, new_ridge_pred)
plt.xlabel("Actual Value")
plt.ylabel("Predicted Value")
plt.suptitle("Ridge Model")
plt.show()

print("New Ridge RMSE:", rmse(new_ridge_pred, y_train))
print("New Ridge Training Accuracy:", new_ridge_model.score(X_train[decent_features], y_train))

**Lasso Regression uses a concept called shrinkage to essentially perform feature selection through variable elimination. Shrinkage is the model’s ability to “shrink” all data points based on how similar they are to a central value. Each data point is weighed against the central value, shrinking towards 0 if the value of the data point is distant from the central value. As data points are “shrunk”, features are passively eliminated through their eventual disappearance of associated data. Lasso Regression is useful when the data exhibits high dimensioniality and high correlation. However, in circumstances where neither of those hold, Lasso is not very useful.**

In [ ]:
lasso_cross_val_model = LassoCV(cv = 5, max_iter = 10000)
lasso_cross_val_fitted = lasso_cross_val_model.fit(X_train, y_train)

print("LassoCV RMSE:", rmse(lasso_cross_val_model.predict(X_train), y_train))
print("LassoCV Training Accuracy: %f" %lasso_cross_val_model.score(X_train, y_train))

In [ ]:
decision_tree = tree.DecisionTreeClassifier(random_state = 10)
decision_tree_model = decision_tree.fit(X_train[decent_features], y_train)
decision_tree_pred = decision_tree_model.predict(X_train[decent_features])

plt.figure(figsize = (8, 6))
plt.scatter(y_train, decision_tree_pred)
plt.xlabel("Actual Value")
plt.ylabel("Predicted Value")
plt.suptitle("Decision Tree Model")
plt.show()

print("Decision Tree RMSE:", rmse(decision_tree_pred, y_train))
print("Decision Tree Training Accuracy:", decision_tree_model.score(X_train, y_train))

In [ ]:
new_decision_tree = tree.DecisionTreeClassifier(random_state = 10)
new_decision_tree_model = new_decision_tree.fit(X_train, y_train)
new_decision_tree_pred = new_decision_tree_model.predict(X_train)

plt.figure(figsize = (8, 6))
plt.scatter(y_train, new_decision_tree_pred)
plt.xlabel("Actual Value")
plt.ylabel("Predicted Value")
plt.suptitle("New Decision Tree Model")
plt.show()

print("New Decision Tree RMSE:", rmse(new_decision_tree_pred, y_train))
print("New Decision Tree Training Accuracy:", new_decision_tree_model.score(X_train[decent_features], y_train))

**In Support Vector Machine Regression, the goal is to find a hyperplane which accurately separates a majority of the data points, creating “classes” based on which side of the hyperplane a data point falls. A hyperplane is a plane which takes on one less dimension than the field it exists in. In order to better understand SVM, consider a line of best fit. In a 2D field, a line of best fit averages the y values to best find a line which either intersects or separates all or most of the data points. If the data is so separated that no line can fit through all points, then the line of best fit becomes a dividing line between data points, categorizing data points based on existing similarities. Now consider the 3D field: a hyperplane is a line of best fit, but in two dimensions, rather than one. SVM is incredibly useful for generalization and is robust to outliers. However, it is very sensitive to noise, as it is difficult to separate the data into classes, making it unideal for certain datasets.**

In [ ]:
# lets try a support vector machine (SVM) regression model

# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html 

# https://scikit-learn.org/stable/modules/svm.html 

SVM_model = make_pipeline(StandardScaler(), SVR(C = 38.42, epsilon = 0.45))

# the argument "SVR" implies that the SVM model is in fact a regression model
# SVR stands for "Support Vector Regression"
# the "C" parameter is the squared L2 regularization penalty paramater in the SVR's underlying implementation.
# As the "C" parameter increases, it takes more time to train the data. Decreasing it leads to more regularization. 
# the "epsilon" parameter represents the distance away from the training loss function, based on the value of C.

SVM_model_fitted = SVM_model.fit(X_train, y_train)
SVM_model_pred = SVM_model_fitted.predict(X_train)

plt.figure(figsize = (14, 10))
plt.scatter(y_train, SVM_model_pred)
plt.xlabel("Actual Value")
plt.ylabel("Predicted Value")
plt.title("Support Vector Machine Regression (SVR) Model",  fontdict = {'fontsize': 20})
plt.show()

print("SVM Model RMSE:", rmse(SVM_model_pred, y_train))
print("SVM Model Training Accuracy:", SVM_model.score(X_train, y_train))

SVM_model.get_params()

In [ ]:
SVM_new_model = make_pipeline(StandardScaler(), SVR(C = 38.42, epsilon = 0.45))

SVM_new_model_fitted = SVM_model.fit(X_train[decent_features], y_train)
SVM_new_model_pred = SVM_model_fitted.predict(X_train[decent_features])

plt.figure(figsize = (14, 10))
plt.scatter(y_train, SVM_new_model_pred)
plt.xlabel("Actual Value")
plt.ylabel("Predicted Value")
plt.title("Support Vector Machine Regression (SVR) Model",  fontdict = {'fontsize': 20})
plt.show()

print("SVM Model RMSE:", rmse(SVM_model_pred, y_train))
print("SVM Model Training Accuracy:", SVM_model.score(X_train, y_train))

SVM_model.get_params()

**We chose SVM as our final model because it gave the lowest rmse, at 2.635, of all our tested models. Root Mean Square Error essentially calculates how far each data point is from the line of best fit. Therefore, a lower rmse shows smaller residuals which means a more accurate model. Furthermore, SVM’s model score showcases the model’s higher training accuracy when compared to that of the other models. SVM’s score was approximately 76%, while other model scores were as low as 20%. Finally, SVM is able to better control for overfitting by utilizing more training data efficiently.**